# preloading

In [ ]:
import os
from pathlib import Path
import datetime
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import pandas as pd
import functools
import itertools

# os.environ["PYLAB_DB_LOCAL"] = ""
# os.environ["PYLAB_DB_OUT"] = ""
import pyflexlab
import pyomnix.pltconfig.color_preset as colors
from pyomnix.utils import CM_TO_INCH, next_lst_gen, constant_generator
from pyflexlab import DataProcess 
from pyomnix import DataManipulator 

In [ ]:
project_name = "Date-Material"
folder = DataProcess(project_name)
plotobj = DataManipulator(11)

In [ ]:
folder.tree()

### Colors Palette

In [ ]:
#plotobj.gui_pan_color()

## File Inspection

In [ ]:
list(plotobj.proj_path.glob("*"))

In [ ]:
os.startfile(plotobj.proj_path)  # open the explorer window

# Colormap
### Create Cmap
```python
mycmap = ListedColormap(["red", "blue", "green"])  # discrete colormap
mycmap = LinearSegmentedColormap.from_list("custom", [(0, "red"), (0.5, "blue"), (1, "green")], N = 256)  # Linear colormap (default to 256, continuous effect)
colormaps.get_cmap(mycmap)  # plot the cmap to show the effect
```
### Modify Existing Cmap
##### Reverse
reverse any existing colormap by appending `_r` to its name or using the `.reversed()` method.
##### Truncate a Cmap
use `constants.truncate_cmap(cmap, min_val, max_val, n=256)`
##### Combine Cmaps
use `constants.combine_cmap(cmap_lst, segment=128)`

#### Common Issues
`(df1[col1] == val1) & (df2[col2] == val2)`, the parentheses here are necessary

## Plotting Template

### load files
```python
for ivg in gate_lst:
    try: # use try to allow vague search
        df_loaded = plotobj.load_dfs(mod_tuple, *vars, tmpfolder="")
        loop = plotobj.identify_direction(df_loaded, "B")  # add direction label
        pos = loop[loop["direction"] == 1]
        neg = loop[loop["direction"] == -1]
        plotobj.load_dfs(idx, pos, f"rhpos-vg{ivg}")
        plotobj.load_dfs(idx+1, neg, f"rhpos-vg{ivg}")
        idx += 2
    except:
        pass
```
### plot data 
```python
fig, ax, para = plotobj.init_canvas(3,3 ,10*5, 6.28*5, lines_per_fig=10)
plotobj.set_unit({"I":"uA"})

axtwin = ax.twinx()
# params[int, int, int] is multidimension-array-like, params[int] is flattened-array-like
params[0,0,0].update(label="")  # could also use one int to index the params._flatten

line = ax.plot(x, y,**params[0,0,0])
line2 = axtwin.plot(x, y,**params[0,0,1])

ax[idx, idy].set_xlabel(rf"$I$ ({plotobj.unit_texname("I")})")
ax.flatten()[idx].set_ylabel(rf"$V$ ({plotobj.unit_texname("V")})")
axtwin.set_ylabel(rf"$V$ ({plotobj.unit_texname("V")})")
plotobj.paint_colors_twin_axes(ax_left=ax, color_left="r", ax_right=axtwin, color_right="b")
# line is a list, so use addition to combine all of them
ax.legend(handles = line+line2, labels=[i.get_label() for i in line+line2],loc = "best", prop=plotobj.legend_font)
ax.set_title(r"")
```

### For multiple lines
- use addition to add lines to a lst for legend management (`line_lst += ax.plot(...)`)
- use `cmap = LinearSegmentedColormap.from_list(...)` to get cmap, and use `cmap(idx)` to get the color
- modify the params length in `init_canvas`, and modify parameters for each line in params

### Contourf
`tricontourf` is for not aligned contour (1-D arrays)
`contourf` is for aligned contour (2-D meshes)
```python
norm = mpl.colors.Normalize(vmin=-2E-8, vmax=2E-8)  # make white to be exactly 0
tcf = ax.tricontourf(x_arr, y_arr, z_arr, levels=128, cmap="seismic", norm=norm)
cb = fig.colorbar(tcf)
cb.set_label(rf"$I$ (%)")
cb.set_ticks(np.linspace(-0.1,0.1,11))
ax.set_xlabel("B (T)")
ax.set_ylabel("G (V)")
```